In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, hour, dayofweek, count, window, lit
from pyspark.ml.feature import StringIndexer, StandardScaler, VectorAssembler, Imputer
from pyspark.ml import Pipeline
import logging
import os

In [14]:
project_root = "/mnt/c/Users/Jason/Desktop/full_ml"  # 👈 CHANGE THIS!
jars_dir = os.path.join(project_root, "jars-extra")
jar_files = [
    "spark-sql-kafka-0-10_2.12-3.5.0.jar",
    "hadoop-aws-3.3.4.jar",
    "aws-java-sdk-bundle-1.12.262.jar"
]
jars_path = ",".join([os.path.join(jars_dir, jar) for jar in jar_files])

In [17]:
logging.getLogger("py4j").setLevel(logging.WARNING)
logging.getLogger("pyspark").setLevel(logging.WARNING)

spark = SparkSession.builder \
    .appName("MinIO-1") \
    .config("spark.jars", jars_path) \
    .config("spark.hadoop.fs.s3a.access.key", "minio") \
    .config("spark.hadoop.fs.s3a.secret.key", "minio123") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

# Set Spark log level to WARN (hides INFO/debug logs)
spark.sparkContext.setLogLevel("WARN")


25/10/10 12:16:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [18]:
df = spark.read.parquet("s3a://mlflow-bucket/transactions/")
df.show()

25/10/10 12:16:42 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://mlflow-bucket/transactions/.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2737)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3569)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3612)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:172)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3716)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3667)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:557)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:366)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:55)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveR

Py4JJavaError: An error occurred while calling o82.parquet.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2737)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3569)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3612)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:172)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3716)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3667)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:557)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:366)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:777)
	at scala.collection.immutable.List.map(List.scala:247)
	at scala.collection.immutable.List.map(List.scala:79)
	at org.apache.spark.sql.execut